## LROSE Blaze Starter Kit Julia Notebook
This is a simple tutorial in a Jupyter notebook for users to get familiar with LROSE Blaze

### Prerequisites

First, make sure that [Jupyter notebook](http://jupyter.readthedocs.io/en/latest/install.html) is installed.

### Workflow 

This starter kit covers a basic LROSE workflow:
1. Print data header from a NEXRAD file using RadxPrint.
2. Convert Level II data to CfRadial format using RadxConvert.
3. Display the data with CfRadial files using Hawkeye.
4. Perform coordinate transformations from the polar grid to a regular grid using Radx2Grid.

Some notes for users:
- RadxConvert also supports the conversion from Level II data to sweep format which can still be utilized by soloii users. 
- RadxConvert will name and sort the files chronologically, so we encourage users to not move the data from the original directory or rename the files.
- Users are encouraged to compile their CfRadial files as an aggregation of sweep files that make up a single volume scan.

#### The following tutorial highlights how to perform the commands, but does not cover all the options. Please see the evolving [documentation](https://nsf-lrose.github.io/docs.html) on the LROSE website for more details

Install
-------

-   Complete the free [registration](../../../software.html) and follow
    the install instructions

If you've already registered, click the link at the bottom of the registration page to take you directly to the software links.

Get some data
-------------

-   A sample NEXRAD level II file is included in the starter kit, or available
    [here](../../../assets/Level2_KAMX_20161006_1906.ar2v) for download with
    this quick start notebook

If you have your own data, just replace the filename in all the
subsequent examples with your data

Inspect the data
----------------

LROSE can handle 24 radar and lidar formats (and counting). To see if
your data works with LROSE, first use RadxPrint to print the metadata.

The following instructions assume you are using the Virtual Toolbox and
the sample NEXRAD file. For native apps installed in your executable
path, just drop 'lrose -- ' or replace with the absolute path to the
location where the binaries are installed. If you are using your own
data, replace the name of the file.

Since we are using a virtual machine, we have to explicitly tell the commands where the data is located. In the notebook, we can define our path as a programming variable. This is slightly different than working in a terminal where environmental variables can be used for this purpose.


In [1]:
path = pwd();

In [2]:
run(`lrose -- RadxPrint -f $path/Level2_KAMX_20161006_1906.ar2v`)

NetCDF: Unknown file format
NetCDF: Unknown file format
=============== RadxVol ===============
  version: 
  title: 
  institution: 
  references: 
  source: ARCHIVE 2 data
  history: 
  comment: 
  volNum: 285
  scanName: Surveillance
  scanId(VCP): 212
--------------- RadxPlatform ---------------
  instrumentName: KAMX
  siteName: 
  instrumentType: radar
  platformType: fixed
  primaryAxis: axis_z
  latitudeDeg: 25.6111
  longitudeDeg: -80.4127
  altitudeKm: 0.033
  sensorHtAglM: 29
  frequencyHz: 2.895e+09
  radarAntGainDbH: 45.3
  radarAntGainDbV: 45.3
  radarBeamWidthDegH: 0.93
  radarBeamWidthDegV: 0.93
  radarReceiverBandwidthMhz: -9999
--------------------------------------------
  rhiMode? N
  startTimeSecs: 2016/10/06 19:07:50
  startNanoSecs: 6e+06
  endTimeSecs: 2016/10/06 19:13:39
  endNanoSecs: 6.79e+08
  ray times are in increasing order
  n sweeps: 14
  n rays: 6120
  n calibs: 1
  n fields: 8
    field[0]: REF
    field[1]: VEL
    field[2]: SW
    field[3]: ZDR
    

If the program returns 'File format not recognized' then it is not a
file format that LROSE currently supports. Otherwise, you should see
text output with all the metadata from your file.

If you prefer to store the printed information in a .txt file, use the command:

In [3]:
run(pipeline(`lrose -- RadxPrint -f $path/Level2_KAMX_20161006_1906.ar2v` , stdout="$path/Level2_KAMX_20161006_1906_info.txt"))

###  Convert the NEXRAD Level II data to different format

The LROSE workflow relies on the CfRadial format for all subsequent
display and analysis. Use RadxConvert to convert your file(s) to
CfRadial. It will work on any file that is recognized by RadxPrint from
the first step through the Radx engine backend.

The default output format is CfRadial, but some other write formats are supported, like Dorade sweepfiles. If you don't choose any command line options RadxConvert will convert to CfRadial.

In [4]:
run(`lrose -- RadxConvert -f $path/Level2_KAMX_20161006_1906.ar2v -outdir $path/output`)

Program 'RadxConvert'
Run-time 2018/07/02 20:10:48.

Copyright (c) 1992 - 2018
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as not to 

There are many more options available for RadxConvert. To use more options you can create a parameter file that can be used for subsequent conversion steps:

In [5]:
run(pipeline(`lrose -- RadxConvert -f -print_params` , stdout="$path/RadxConvert.params"))

If you want to convert a file to Dorade format for use in soloii or solo3, just add the -dorade flag to the RadxConvert command or change the output format in the parameter file. We recommend migrating your workflow to CfRadial, but conversion to Dorade is supported for existing workflows.

In [6]:
run(`lrose -- HawkEye -f $path/output/20161006/cfrad.20161006_190750.006_to_20161006_191339.679_KAMX_Surveillance_SUR.nc`)

libGL error: No matching fbConfigs or visuals found
libGL error: failed to load driver: swrast


### Grid the data into Cartesian space

A common first step for working with NEXRAD data is to interpolate the
data from its native radial coordinate to a regular grid. Radx2Grid can
grid ground-based and some airborne lidar and radar data in spherical
and Cartesian space. While there are many options for Radx2Grid, a basic
gridding technique can be accomplished simply:

In [7]:
run(`lrose -- Radx2Grid -f $path/output/20161006/cfrad.20161006_190750.006_to_20161006_191339.679_KAMX_Surveillance_SUR.nc -outdir $path/grid`)

Program 'Radx2Grid'
Run-time 2018/07/02 20:11:09.

Copyright (c) 1992 - 2018
University Corporation for Atmospheric Research (UCAR)
National Center for Atmospheric Research (NCAR)
Boulder, Colorado, USA.

Redistribution and use in source and binary forms, with
or without modification, are permitted provided that the following
conditions are met:

1) Redistributions of source code must retain the above copyright
notice, this list of conditions and the following disclaimer.

2) Redistributions in binary form must reproduce the above copyright
notice, this list of conditions and the following disclaimer in the
documentation and/or other materials provided with the distribution.

3) Neither the name of UCAR, NCAR nor the names of its contributors, if
any, may be used to endorse or promote products derived from this
software without specific prior written permission.

4) If the software is modified to produce derivative works, such modified
software should be clearly marked, so as not to co

There are many more options available for RadxConvert. To use more options you can create a parameter file that can be used for subsequent conversion steps:

In [8]:
run(pipeline(`lrose -- Radx2Grid -f -print_params` , stdout="$path/Radx2Grid.params"))

### Quick Start Complete!

If you have successfully run the previous steps then you should now have
a Cartesian gridded NetCDF file to use for further analysis called
ncf_20161006_191339.nc in the 'grid' subdirectory. From here, you can
read in the CfRadial or gridded netCDF file using Julia, Python, or
other programming language for further analysis. Continue with the
[documentation](https://nsf-lrose.github.io/docs.html) to find out more about LROSE and the many options and
parameters available in the LROSE-Blaze software suite.
